# KoBERT 기반 뉴스 스탠스 분류 모델 학습

이 노트북은 뉴스 기사를 **옹호(Support)**, **중립(Neutral)**, **비판(Oppose)** 3개 클래스로 분류하는 KoBERT 모델을 학습합니다.

## 목차
1. [환경 설정](#1-환경-설정)
2. [데이터 준비](#2-데이터-준비)
3. [데이터 전처리](#3-데이터-전처리)
4. [모델 정의](#4-모델-정의)
5. [학습](#5-학습)
6. [평가](#6-평가)
7. [모델 저장](#7-모델-저장)

## 1. 환경 설정

필요한 패키지를 설치하고 임포트합니다.

In [ ]:
# Colab 환경 확인
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colab 환경에서 실행 중입니다.")
except:
    IN_COLAB = False
    print("⚠️  로컬 환경에서 실행 중입니다.")

In [ ]:
# 필수 패키지 설치
!pip install -q torch transformers
!pip install -q tokenizers
!pip install -q scikit-learn pandas numpy matplotlib seaborn
!pip install -q tqdm

print("✅ 패키지 설치 완료!")

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, AutoTokenizer

import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import os
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 랜덤 시드 고정
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️  사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")

## 2. 데이터 준비

### 옵션 1: 샘플 데이터로 테스트
실제 데이터가 없을 때 샘플 데이터로 모델을 테스트할 수 있습니다.

In [ ]:
# 샘플 데이터 생성 (학습을 위해 더 많은 샘플 추가)
sample_data = []

# 옹호 (Support) - 레이블 0
support_samples = [
    "정부의 새로운 경제 정책은 경제 성장을 촉진하고 일자리를 창출할 것으로 기대된다. 전문가들은 이번 정책이 시의적절한 조치라고 평가했다.",
    "이번 개혁안은 국민의 삶의 질을 향상시키는 데 크게 기여할 것이다. 여러 경제 지표들이 긍정적인 변화를 보여주고 있다.",
    "정책 시행 후 관련 업계의 매출이 증가하고 있으며, 고용률도 상승세를 보이고 있다. 정책의 효과가 가시화되고 있는 것으로 보인다.",
    "전문가들은 이번 조치가 장기적으로 국가 경쟁력을 높이는 데 도움이 될 것이라고 분석했다. 해외 사례에서도 유사한 성과를 거둔 바 있다.",
    "시민단체들도 이번 정책에 대해 긍정적인 반응을 보이고 있으며, 국민 만족도 조사에서도 높은 점수를 받았다.",
    "이번 법안은 국민의 권리를 보호하고 사회 안전망을 강화하는 데 큰 도움이 될 것으로 보인다.",
    "정책 효과가 실제 통계로 입증되고 있으며, 관련 분야 전문가들도 긍정적인 평가를 내리고 있다.",
    "이번 조치는 국제 사회에서도 모범 사례로 평가받고 있으며, 여러 국가가 벤치마킹을 원하고 있다.",
]

# 중립 (Neutral) - 레이블 1
neutral_samples = [
    "정부가 새로운 정책을 발표했다. 정책의 주요 내용은 경제 활성화와 일자리 창출이다. 시행 시기는 내년 초로 예정되어 있다.",
    "이번 개혁안에 대해 전문가들의 의견이 엇갈리고 있다. 일부는 긍정적으로 평가하는 반면, 다른 전문가들은 신중한 접근이 필요하다고 말했다.",
    "정책 발표 후 관련 업계에서는 다양한 반응이 나오고 있다. 구체적인 시행 방안에 대한 논의가 진행 중이다.",
    "새로운 제도가 도입되면 기존 시스템에 변화가 생길 것으로 예상된다. 정부는 관계 부처 협의를 통해 세부 사항을 조율하고 있다.",
    "이번 조치에 대한 국민 여론은 아직 확정되지 않았다. 향후 추이를 지켜봐야 할 것으로 보인다.",
    "정부가 오늘 새로운 경제 정책을 공식 발표했다. 주요 내용은 다음 주 상세히 공개될 예정이다.",
    "관련 법안이 국회에 제출되었으며, 현재 상임위원회에서 검토 중이다. 통과 시기는 미정이다.",
    "전문가들은 이번 정책의 효과를 판단하기 위해서는 최소 6개월 이상의 시간이 필요하다고 말했다.",
]

# 비판 (Oppose) - 레이블 2
oppose_samples = [
    "정부의 정책은 현실을 제대로 반영하지 못한 채 졸속으로 추진되고 있다. 전문가들은 부작용을 우려하고 있다.",
    "이번 개혁안은 실효성이 의심된다. 과거 유사한 정책들이 실패했던 전례를 고려할 때, 신중한 재검토가 필요하다.",
    "정책 시행으로 인해 중소기업들의 부담이 가중될 것이라는 우려가 제기되고 있다. 업계는 강력히 반발하고 있다.",
    "여러 문제점이 지적되고 있음에도 불구하고 정부는 일방적으로 추진하고 있다. 국민의 목소리를 듣지 않는다는 비판이 나온다.",
    "전문가들은 이번 조치가 오히려 경제에 악영향을 미칠 수 있다고 경고했다. 시장의 반응도 부정적이다.",
    "이번 정책은 준비 부족과 졸속 처리로 인해 혼란만 가중시키고 있다는 비판이 거세다.",
    "야당은 이번 법안이 헌법 정신에 위배된다며 강력히 반대하고 있으며, 철회를 요구하고 있다.",
    "시민단체들은 이번 조치가 국민의 권리를 침해할 수 있다며 우려를 표명하고 있다.",
]

# 데이터 생성
for text in support_samples:
    sample_data.append({"text": text, "label": 0})
for text in neutral_samples:
    sample_data.append({"text": text, "label": 1})
for text in oppose_samples:
    sample_data.append({"text": text, "label": 2})

# DataFrame 생성
sample_df = pd.DataFrame(sample_data)
print(f"\n📊 샘플 데이터 생성 완료: {len(sample_df)}개")
print(f"\n클래스별 분포:")
print(sample_df['label'].value_counts().sort_index())
print(f"\n0: 옹호 (Support)")
print(f"1: 중립 (Neutral)")
print(f"2: 비판 (Oppose)")

### 옵션 2: 실제 데이터 업로드

실제 라벨링된 데이터가 있다면 아래 셀을 실행하여 업로드하세요.

**데이터 형식 (CSV):**
```csv
text,label
"기사 본문 1",0
"기사 본문 2",1
"기사 본문 3",2
```

**데이터 형식 (JSON):**
```json
[
  {"text": "기사 본문 1", "label": 0},
  {"text": "기사 본문 2", "label": 1},
  {"text": "기사 본문 3", "label": 2}
]
```

**레이블:**
- 0: 옹호 (Support)
- 1: 중립 (Neutral)
- 2: 비판 (Oppose)

In [ ]:
# 실제 데이터 업로드 (선택사항)
USE_REAL_DATA = False  # True로 변경하여 실제 데이터 사용

if USE_REAL_DATA:
    if IN_COLAB:
        from google.colab import files
        print("📁 데이터 파일을 업로드하세요 (CSV 또는 JSON)")
        uploaded = files.upload()
        
        # 파일명 가져오기
        filename = list(uploaded.keys())[0]
        
        # 파일 형식에 따라 로드
        if filename.endswith('.csv'):
            df = pd.read_csv(filename)
        elif filename.endswith('.json'):
            df = pd.read_json(filename)
        else:
            raise ValueError("CSV 또는 JSON 파일만 지원됩니다.")
    else:
        # 로컬 환경
        data_path = input("데이터 파일 경로를 입력하세요: ")
        if data_path.endswith('.csv'):
            df = pd.read_csv(data_path)
        elif data_path.endswith('.json'):
            df = pd.read_json(data_path)
        else:
            raise ValueError("CSV 또는 JSON 파일만 지원됩니다.")
    
    print(f"\n✅ 데이터 로드 완료: {len(df)}개")
    print(f"\n클래스별 분포:")
    print(df['label'].value_counts().sort_index())
else:
    # 샘플 데이터 사용
    df = sample_df
    print("\n📝 샘플 데이터를 사용합니다.")

In [ ]:
# 데이터 확인
print("\n=== 데이터 미리보기 ===")
print(df.head())

print("\n=== 데이터 정보 ===")
print(f"전체 샘플 수: {len(df)}")
print(f"\n클래스별 분포:")
label_counts = df['label'].value_counts().sort_index()
for label, count in label_counts.items():
    label_name = ['옹호', '중립', '비판'][label]
    print(f"  {label} ({label_name}): {count}개 ({count/len(df)*100:.1f}%)")

# 클래스 분포 시각화
plt.figure(figsize=(8, 5))
label_names = ['옹호\n(Support)', '중립\n(Neutral)', '비판\n(Oppose)']
plt.bar(range(3), [label_counts.get(i, 0) for i in range(3)])
plt.xticks(range(3), label_names)
plt.ylabel('Count')
plt.title('Class Distribution')
plt.tight_layout()
plt.show()

## 3. 데이터 전처리

KoBERT 토크나이저를 사용하여 텍스트를 토큰화합니다.

In [ ]:
# KoBERT 토크나이저 로드
print("🔄 KoBERT 토크나이저 로딩 중...")
try:
    # AutoTokenizer를 사용하여 KoBERT 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kobert-base-v1',
        trust_remote_code=True
    )
    print("✅ 토크나이저 로드 완료!")
    print(f"   Vocab 크기: {len(tokenizer)}")
except Exception as e:
    print(f"⚠️  오류 발생: {e}")
    print("\n대안: monologg/kobert 토크나이저 사용")
    tokenizer = AutoTokenizer.from_pretrained('monologg/kobert')
    print("✅ 대안 토크나이저 로드 완료!")
    print(f"   Vocab 크기: {len(tokenizer)}")

In [ ]:
# 토크나이저 테스트
test_text = "이것은 테스트 문장입니다."
encoded = tokenizer.encode_plus(
    test_text,
    add_special_tokens=True,
    max_length=50,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

print("\n=== 토크나이저 테스트 ===")
print(f"원본 텍스트: {test_text}")
print(f"토큰화 결과 shape: {encoded['input_ids'].shape}")
print(f"Input IDs (처음 20개): {encoded['input_ids'][0][:20].tolist()}")
print("✅ 토크나이저 정상 작동!")

In [ ]:
# 커스텀 Dataset 클래스
class StanceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

print("✅ Dataset 클래스 정의 완료!")

In [ ]:
# 하이퍼파라미터 설정
MAX_LENGTH = 256  # 최대 시퀀스 길이 (샘플 데이터에는 256으로 충분)
BATCH_SIZE = 4    # 배치 크기 (GPU 메모리에 따라 조정)
LEARNING_RATE = 2e-5  # 학습률
EPOCHS = 10       # 에폭 수 (샘플 데이터가 적으므로 더 많이)
DROPOUT = 0.3     # 드롭아웃 비율
TEST_SIZE = 0.2   # 테스트 데이터 비율
VAL_SIZE = 0.1    # 검증 데이터 비율

print("\n=== 하이퍼파라미터 ===")
print(f"최대 시퀀스 길이: {MAX_LENGTH}")
print(f"배치 크기: {BATCH_SIZE}")
print(f"학습률: {LEARNING_RATE}")
print(f"에폭 수: {EPOCHS}")
print(f"드롭아웃: {DROPOUT}")
print(f"테스트/검증 비율: {TEST_SIZE}/{VAL_SIZE}")

In [ ]:
# Train/Val/Test 분할
# 데이터가 적으므로 stratify 사용하여 클래스 비율 유지
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].values,
    df['label'].values,
    test_size=TEST_SIZE,
    random_state=SEED,
    stratify=df['label'].values
)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts,
    train_labels,
    test_size=VAL_SIZE/(1-TEST_SIZE),
    random_state=SEED,
    stratify=train_labels
)

print("\n=== 데이터 분할 ===")
print(f"학습 데이터: {len(train_texts)}개")
print(f"검증 데이터: {len(val_texts)}개")
print(f"테스트 데이터: {len(test_texts)}개")

# Dataset 생성
train_dataset = StanceDataset(train_texts, train_labels, tokenizer, MAX_LENGTH)
val_dataset = StanceDataset(val_texts, val_labels, tokenizer, MAX_LENGTH)
test_dataset = StanceDataset(test_texts, test_labels, tokenizer, MAX_LENGTH)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print("\n✅ 데이터 로더 생성 완료!")

## 4. 모델 정의

KoBERT 기반 스탠스 분류 모델을 정의합니다.

In [ ]:
class StanceClassifier(nn.Module):
    """
    KoBERT 기반 스탠스 분류 모델
    
    Classes:
        0: 옹호 (Support)
        1: 중립 (Neutral)
        2: 비판 (Oppose)
    """
    
    def __init__(self, n_classes=3, dropout=0.3):
        super(StanceClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('skt/kobert-base-v1')
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # [CLS] 토큰의 출력 사용
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        return logits

# 모델 초기화
print("🔄 모델 로딩 중...")
model = StanceClassifier(n_classes=3, dropout=DROPOUT)
model = model.to(device)

# 모델 파라미터 수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✅ 모델 로드 완료!")
print(f"   전체 파라미터: {total_params:,}")
print(f"   학습 가능한 파라미터: {trainable_params:,}")

## 5. 학습

모델을 학습하고 검증합니다.

In [ ]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# 학습률 스케줄러 (선택사항)
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

# 학습 기록 저장용
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print("\n=== 학습 시작 ===")

In [ ]:
# 학습 함수
def train_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    progress_bar = tqdm(data_loader, desc='Training')
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        
        loss.backward()
        # 그래디언트 클리핑 (선택사항)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100 * correct / total:.2f}%'
        })
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct / total
    
    return avg_loss, accuracy

# 검증 함수
def eval_model(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct / total
    
    return avg_loss, accuracy

print("✅ 학습/검증 함수 정의 완료!")

In [ ]:
# 학습 루프
best_val_acc = 0
best_model_state = None
patience_counter = 0
max_patience = 5

for epoch in range(EPOCHS):
    print(f"\n{'='*50}")
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print(f"{'='*50}")
    
    # 학습
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    print(f"\n📊 Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%")
    
    # 검증
    val_loss, val_acc = eval_model(model, val_loader, criterion, device)
    print(f"📊 Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
    
    # 학습률 스케줄러 업데이트
    scheduler.step(val_loss)
    
    # 기록 저장
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # 최고 성능 모델 저장
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
        patience_counter = 0
        print(f"\n⭐ 새로운 최고 성능! (Val Acc: {best_val_acc*100:.2f}%)")
    else:
        patience_counter += 1
        if patience_counter >= max_patience:
            print(f"\n⚠️  Early stopping: {max_patience} 에폭 동안 개선 없음")
            break

print(f"\n\n{'='*50}")
print(f"🎉 학습 완료!")
print(f"최고 검증 정확도: {best_val_acc*100:.2f}%")
print(f"{'='*50}")

# 최고 성능 모델로 복원
if best_model_state:
    model.load_state_dict(best_model_state)

In [ ]:
# 학습 과정 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss 그래프
ax1.plot(history['train_loss'], label='Train Loss', marker='o')
ax1.plot(history['val_loss'], label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True)

# Accuracy 그래프
ax2.plot([acc*100 for acc in history['train_acc']], label='Train Acc', marker='o')
ax2.plot([acc*100 for acc in history['val_acc']], label='Val Acc', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## 6. 평가

테스트 데이터셋으로 모델 성능을 평가합니다.

In [ ]:
# 테스트 데이터 평가
print("\n=== 테스트 평가 ===")
test_loss, test_acc = eval_model(model, test_loader, criterion, device)
print(f"\n📊 Test Loss: {test_loss:.4f}")
print(f"📊 Test Accuracy: {test_acc*100:.2f}%")

In [ ]:
# 상세 분류 보고서
def get_predictions(model, data_loader, device):
    model.eval()
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc='Getting predictions'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask)
            _, preds = torch.max(outputs, 1)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    return np.array(predictions), np.array(true_labels)

# 예측값 가져오기
predictions, true_labels = get_predictions(model, test_loader, device)

# 분류 보고서
target_names = ['옹호 (Support)', '중립 (Neutral)', '비판 (Oppose)']
print("\n=== 분류 보고서 ===")
print(classification_report(true_labels, predictions, target_names=target_names, digits=4, zero_division=0))

In [ ]:
# Confusion Matrix 시각화
cm = confusion_matrix(true_labels, predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Support', 'Neutral', 'Oppose'],
            yticklabels=['Support', 'Neutral', 'Oppose'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# 정규화된 Confusion Matrix
cm_normalized = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-10)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=['Support', 'Neutral', 'Oppose'],
            yticklabels=['Support', 'Neutral', 'Oppose'])
plt.title('Normalized Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

## 7. 모델 저장

학습된 모델을 저장하고 다운로드합니다.

In [ ]:
# 저장 디렉토리 생성
save_dir = 'saved_models'
os.makedirs(save_dir, exist_ok=True)

# 모델 저장
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_filename = f'stance_classifier_{timestamp}.pth'
model_path = os.path.join(save_dir, model_filename)

torch.save(model.state_dict(), model_path)
print(f"\n✅ 모델 저장 완료: {model_path}")

# 학습 메타데이터 저장
metadata = {
    'timestamp': timestamp,
    'test_accuracy': float(test_acc),
    'test_loss': float(test_loss),
    'best_val_accuracy': float(best_val_acc),
    'hyperparameters': {
        'max_length': MAX_LENGTH,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'epochs': EPOCHS,
        'dropout': DROPOUT
    },
    'data_info': {
        'total_samples': len(df),
        'train_samples': len(train_texts),
        'val_samples': len(val_texts),
        'test_samples': len(test_texts)
    },
    'model_name': 'skt/kobert-base-v1'
}

metadata_filename = f'metadata_{timestamp}.json'
metadata_path = os.path.join(save_dir, metadata_filename)

with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"✅ 메타데이터 저장 완료: {metadata_path}")

In [ ]:
# Colab에서 파일 다운로드
if IN_COLAB:
    from google.colab import files
    
    print("\n📥 모델 다운로드 중...")
    files.download(model_path)
    files.download(metadata_path)
    print("✅ 다운로드 완료!")
else:
    print(f"\n💾 로컬 환경에서는 다음 경로에 저장되었습니다:")
    print(f"   모델: {model_path}")
    print(f"   메타데이터: {metadata_path}")

## 8. 테스트 예측

저장된 모델로 실제 예측을 테스트해봅니다.

In [ ]:
# 예측 함수
def predict_stance(text, model, tokenizer, device, max_length=256):
    model.eval()
    
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probabilities = torch.softmax(outputs, dim=1)
        confidence, predicted_class = torch.max(probabilities, dim=1)
    
    stance_labels = {0: "옹호", 1: "중립", 2: "비판"}
    
    return {
        'stance': stance_labels[predicted_class.item()],
        'stance_id': predicted_class.item(),
        'confidence': round(confidence.item(), 4),
        'probabilities': {
            stance_labels[i]: round(prob, 4)
            for i, prob in enumerate(probabilities[0].tolist())
        }
    }

# 테스트 예측
test_texts_sample = [
    "이번 정책은 경제 발전에 큰 도움이 될 것으로 기대된다. 전문가들도 긍정적으로 평가하고 있다.",
    "정부가 새로운 법안을 발표했다. 구체적인 시행 방안은 추후 공개될 예정이다.",
    "이번 조치는 국민에게 부담만 가중시킬 뿐이다. 여러 문제점이 지적되고 있다."
]

print("\n=== 예측 테스트 ===")
for i, text in enumerate(test_texts_sample, 1):
    result = predict_stance(text, model, tokenizer, device, MAX_LENGTH)
    print(f"\n{i}. 텍스트: {text}")
    print(f"   예측: {result['stance']} (신뢰도: {result['confidence']*100:.2f}%)")
    print(f"   확률: 옹호={result['probabilities']['옹호']:.4f}, "
          f"중립={result['probabilities']['중립']:.4f}, "
          f"비판={result['probabilities']['비판']:.4f}")

## 9. 다음 단계

### 실제 데이터로 학습하기

1. **데이터 준비**
   - `data/DATA_PREPARATION_GUIDE.md` 참고
   - 최소 클래스당 100개 이상 권장
   - CSV 또는 JSON 형식으로 준비

2. **노트북 재실행**
   - "옵션 2: 실제 데이터 업로드" 섹션에서 `USE_REAL_DATA = True`로 변경
   - 데이터 업로드 후 학습 진행

3. **하이퍼파라미터 튜닝**
   - `MAX_LENGTH`: 512 (긴 기사용)
   - `BATCH_SIZE`: 8-16 (GPU 메모리에 따라)
   - `EPOCHS`: 5-10
   - `LEARNING_RATE`: 1e-5 ~ 5e-5

4. **FastAPI 서버 통합**
   - 학습된 모델(.pth)을 `saved_models/` 디렉토리에 복사
   - `.env` 파일에 모델 경로 설정
   - API 서버 재시작

### 모델 성능 개선 팁

- 데이터 증강 (Augmentation)
- 클래스 불균형 처리 (Weighted Loss)
- 앙상블 모델
- Fine-tuning 전략 변경 (BERT layer freezing)

### 문의

문제가 발생하면 GitHub Issues에 올려주세요!